In [17]:
def read_mesh(filename):
    """ read XYZ for each vertex.
    """
    assert os.path.isfile(filename)
    with open(filename, 'rb') as f:
        plydata = plyfile.PlyData.read(f)
        num_verts = plydata['vertex'].count
        vertices = np.zeros(shape=[num_verts, 8], dtype=np.float32)
        vertices[:, 0] = plydata['vertex'].data['x']
        vertices[:, 1] = plydata['vertex'].data['y']
        vertices[:, 2] = plydata['vertex'].data['z']
        vertices[:, 3] = plydata['vertex'].data['red']
        vertices[:, 4] = plydata['vertex'].data['green']
        vertices[:, 5] = plydata['vertex'].data['blue']
        vertices[:, 6] = plydata['vertex'].data['alpha']
        vertices[:, 7] = plydata['vertex'].data['label']

    return vertices, plydata['face']

def write_mesh(vertices, faces):
    new_vertices = []
    for i in range(vertices.shape[0]):
        new_vertices.append(
            (
                vertices[i][0],
                vertices[i][1],
                vertices[i][2],
                vertices[i][3],
                vertices[i][4],
                vertices[i][5],
                vertices[i][6],
                vertices[i][7],
            )
        )

    vertices = np.array(
        new_vertices,
        dtype=[
            ("x", np.dtype("float32")),
            ("y", np.dtype("float32")),
            ("z", np.dtype("float32")),
            ("red", np.dtype("uint8")),
            ("green", np.dtype("uint8")),
            ("blue", np.dtype("uint8")),
            ("alpha", np.dtype("uint8")),
            ("label", np.dtype("ushort"))
        ]
    )

    vertices = plyfile.PlyElement.describe(vertices, "vertex")
    mesh = plyfile.PlyData([vertices, faces])
    mesh.write("./aligned_mesh1.ply")


In [18]:
import os
import plyfile
import numpy as np

filename = os.path.join("D:/aaaaaaaaaaaaaaaaa/23-24/astar/1-2data/ScanNet/scannet/scans/scene0000_00/scene0000_00_vh_clean_2.labels.ply")
vertices, faces = read_mesh(filename)


meta_file = os.path.join("D:/aaaaaaaaaaaaaaaaa/23-24/astar/1-2data/ScanNet/scannet/scans/scene0000_00/scene0000_00.txt")
lines = open(meta_file).readlines()
axis_align_matrix = None
for line in lines:
    if 'axisAlignment' in line:
        axis_align_matrix = [float(x) for x in line.rstrip().strip('axisAlignment = ').split(' ')]


if axis_align_matrix != None:
    axis_align_matrix = np.array(axis_align_matrix).reshape((4, 4))
    pts = np.ones((vertices.shape[0], 4))
    pts[:, 0:3] = vertices[:, :3]
    pts = np.dot(pts, axis_align_matrix.transpose())
    aligned_vertices = np.copy(vertices)
    aligned_vertices[:, 0:3] = pts[:, 0:3]

write_mesh(aligned_vertices, faces)
